<a href="https://www.kaggle.com/code/sohailshaik272/csv-to-parquet-pickle-amex?scriptVersionId=105696793" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

**Apache Arrow is a development platform for in-memory analytics. It contains a set of technologies that enable big data systems to store, process and move data fast.**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyarrow as pa
import pyarrow.parquet as pq
import pickle
#Apache Arrow is a development platform for in-memory analytics. 
#It contains a set of technologies that enable big data systems to store, process and move data
#fast
import scipy
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


In [2]:
train_path = '/kaggle/input/amex-default-prediction/train_data.csv'
test_path = '/kaggle/input/amex-default-prediction/test_data.csv'
parquet_file_train = 'train_amex.parquet'
parquet_file_test = 'test_amex.parquet'
pickle_file_train = 'train_amex_pickle.pkl'
pickle_file_test = 'test_amex_pickle.pkl'
train = pd.read_csv(train_path, nrows=1)
features = train.columns.tolist()

In [3]:
pd.set_option('display.max_rows', None)
features

['customer_ID',
 'S_2',
 'P_2',
 'D_39',
 'B_1',
 'B_2',
 'R_1',
 'S_3',
 'D_41',
 'B_3',
 'D_42',
 'D_43',
 'D_44',
 'B_4',
 'D_45',
 'B_5',
 'R_2',
 'D_46',
 'D_47',
 'D_48',
 'D_49',
 'B_6',
 'B_7',
 'B_8',
 'D_50',
 'D_51',
 'B_9',
 'R_3',
 'D_52',
 'P_3',
 'B_10',
 'D_53',
 'S_5',
 'B_11',
 'S_6',
 'D_54',
 'R_4',
 'S_7',
 'B_12',
 'S_8',
 'D_55',
 'D_56',
 'B_13',
 'R_5',
 'D_58',
 'S_9',
 'B_14',
 'D_59',
 'D_60',
 'D_61',
 'B_15',
 'S_11',
 'D_62',
 'D_63',
 'D_64',
 'D_65',
 'B_16',
 'B_17',
 'B_18',
 'B_19',
 'D_66',
 'B_20',
 'D_68',
 'S_12',
 'R_6',
 'S_13',
 'B_21',
 'D_69',
 'B_22',
 'D_70',
 'D_71',
 'D_72',
 'S_15',
 'B_23',
 'D_73',
 'P_4',
 'D_74',
 'D_75',
 'D_76',
 'B_24',
 'R_7',
 'D_77',
 'B_25',
 'B_26',
 'D_78',
 'D_79',
 'R_8',
 'R_9',
 'S_16',
 'D_80',
 'R_10',
 'R_11',
 'B_27',
 'D_81',
 'D_82',
 'S_17',
 'R_12',
 'B_28',
 'R_13',
 'D_83',
 'R_14',
 'R_15',
 'D_84',
 'R_16',
 'B_29',
 'B_30',
 'S_18',
 'D_86',
 'D_87',
 'R_17',
 'R_18',
 'D_88',
 'B_31',
 'S_


#### categorical variables
['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

* B_30 -> uint8   (0 to 2)
* B_38 -> uint8   (1 to 7)
* D_114 -> binary (0 or 1)
* D_116 -> binary (0 or 1)
* D_117 -> int8   (-1 to 7)
* D_120 -> binary (0 or 1)
* D_126 -> int8 (-1 to 1)
* D_63 -> object
* D_64 -> object
* D_66 -> binary (proper) (1) has 70% missing value
* D_68 -> unint8 (1 to 6)

**Other columns** 
* customer_ID - object
* S_2 - object
* B_31(int64) - binary

**Rest all columns change float64 to float32**


In [4]:
dtypes = {'customer_ID': 'str',
       'S_2': 'str',
       'D_63': 'str',
       'D_64': 'str'
                  }
for col in features:
    if col not in dtypes.keys():
        dtypes[col] = 'float32'

**Why float32 is better than float16, check it out here**
https://discourse.julialang.org/t/massive-performance-penalty-for-float16-compared-to-float32/6864/3

In [5]:
def write_parquet(path, feats, save_loc, chunk_size=100000):
    schema = ''
    writer = ''
    if len(feats) > 0:
        for i,chunk in enumerate(pd.read_csv(path, dtype=dtypes, iterator=True, chunksize=100000)):
            if i == 0:
                schema = pa.Table.from_pandas(df=chunk).schema
                writer = pq.ParquetWriter(save_loc, schema, compression='snappy')
            table = pa.Table.from_pandas(chunk, schema=schema)
            writer.write_table(table)
    return None

In [6]:

def write_pickle(path, feats,save_loc, chunck_size = 100000):
    with open(save_loc, 'wb') as f:
        for i,chunk in enumerate(pd.read_csv(path, dtype=dtypes, iterator=True, chunksize=100000)):
            pickle.dump(chunk, f)
                

In [7]:
start_train = time.time()
write_parquet(train_path, features, parquet_file_train)
end_train = time.time()
start_test = time.time()
write_parquet(test_path, features, parquet_file_test)
end_test = time.time()
minutes, seconds = divmod(end_train-start_train, 60)
print(f"Time taken for converting train.csv to parquet: {minutes} minutes and {seconds} seconds....")
minutes_test, seconds_test = divmod(end_test-start_test, 60)
print(f"Time taken for converting train.csv to parquet: {minutes_test} minutes and {seconds_test} seconds....")

Time taken for converting train.csv to parquet: 7.0 minutes and 52.92708349227905 seconds....
Time taken for converting train.csv to parquet: 16.0 minutes and 6.080052614212036 seconds....


In [8]:
#%%time
#write_pickle(train_path,features,pickle_file_train)

In [9]:
#%%time
#write_pickle(test_path,features,pickle_file_test)